In [1]:
#export
tableName = 'dynamodbDemoVilla-LogTable-1U6Z3WLB40Q82'
functionName = 'dynamodbDemoVilla-IngestFunction-UHL6PMPDPV9A'
s3bucketName = 'dynamodbdemovilla-ingestbucket-b6avfx3ejcop'

In [2]:
#export
from pynamodb.models import Model
from pynamodb.attributes import (
    UnicodeAttribute, NumberAttribute, UnicodeSetAttribute, UTCDateTimeAttribute, JSONAttribute
)
from s3bz.s3bz import S3
from awsSchema.apigateway import Event, Response
import json

In [3]:
#export
# from pynamodb.indexes import GlobalSecondaryIndex, AllProjection
# from pynamodb.attributes import NumberAttribute


# class DateIndex(GlobalSecondaryIndex):
#     """
#     This class represents a global secondary index
#     """
#     class Meta:
#         index_name = 'foo-index'
#         read_capacity_units = 1
#         write_capacity_units = 1
#         projection = AllProjection()

#     # This attribute is the hash key for the index
#     # Note that this attribute must also exist
#     # in the model
#     view = NumberAttribute(default=0, hash_key=True)

# Define database structure

In [14]:
#export
class ExampleData(Model):
    class Meta:
      table_name = tableName
      region = 'ap-southeast-1'
    transaction_id = UnicodeAttribute(hash_key=True)
    date = NumberAttribute()
    data = JSONAttribute()
    @classmethod
    def fromDict(cls, input_):
      return cls(
        transaction_id = input_['id_'],
        date = input_['date'],
        data = input_
         )


In [15]:
inputExample = {
    "id_": "123",
    "date": 1234323536,
    "merchant_id": "32434",
    "merchant_name": "kdsjfljsldjf",
    "transaction_id": "qfeef",
    "amount": "qefqf",
    "transaction_type": "feqfe",
    "consumer_name": "fewqfq",
    "mobile": "qfefeq",
    "terminal_id": "qfewqf",
    "transaction_date_time": "fqef"
}

## save data

In [16]:
data = ExampleData.fromDict(inputExample)
data.save()

{'ConsumedCapacity': {'CapacityUnits': 1.0,
  'TableName': 'dynamodbDemoVilla-LogTable-1U6Z3WLB40Q82'}}

## Query data

In [17]:
next(ExampleData.query('123')).data

{'id_': '123',
 'date': 1234323536,
 'merchant_id': '32434',
 'merchant_name': 'kdsjfljsldjf',
 'transaction_id': 'qfeef',
 'amount': 'qefqf',
 'transaction_type': 'feqfe',
 'consumer_name': 'fewqfq',
 'mobile': 'qfefeq',
 'terminal_id': 'qfewqf',
 'transaction_date_time': 'fqef'}

## save to s3

In [18]:
inputExamples = [{
    "id_": "123",
    "date": 1234323536,
    "merchant_id": "32434",
    "merchant_name": "kdsjfljsldjf",
    "transaction_id": "qfeef",
    "amount": "qefqf",
    "transaction_type": "feqfe",
    "consumer_name": "fewqfq",
    "mobile": "qfefeq",
    "terminal_id": "qfewqf",
    "transaction_date_time": "fqef"
  },
  {
    "id_": "123234",
    "date": 1234323536,
    "merchant_id": "32434",
    "merchant_name": "kdsjfljsldjf",
    "transaction_id": "qfeef",
    "amount": "qefqf",
    "transaction_type": "feqfe",
    "consumer_name": "fewqfq",
    "mobile": "qfefeq",
    "terminal_id": "qfewqf",
    "transaction_date_time": "fqef"
}]
  

In [19]:
def saveToS3(data, inputPath, key):
  with open (inputPath, 'w') as f:
    json.dump(data, f)
  S3.saveFile(key =key ,path = inputPath,bucket = s3bucketName)

In [20]:
saveToS3(inputExamples, '/tmp/data', 'test')

In [21]:
#export
S3.loadFile(key = 'test', path = '/tmp/loaded',bucket=s3bucketName)
with open('/tmp/loaded', 'r') as f:
  print(f.read())

[{"id_": "123", "date": 1234323536, "merchant_id": "32434", "merchant_name": "kdsjfljsldjf", "transaction_id": "qfeef", "amount": "qefqf", "transaction_type": "feqfe", "consumer_name": "fewqfq", "mobile": "qfefeq", "terminal_id": "qfewqf", "transaction_date_time": "fqef"}, {"id_": "123234", "date": 1234323536, "merchant_id": "32434", "merchant_name": "kdsjfljsldjf", "transaction_id": "qfeef", "amount": "qefqf", "transaction_type": "feqfe", "consumer_name": "fewqfq", "mobile": "qfefeq", "terminal_id": "qfewqf", "transaction_date_time": "fqef"}]


In [22]:
#export
from pprint import pprint
def ingest(event, *args):
  # set up 
  body = Event.parseBody(event)
  key = body['key']
  path = '/tmp/loaded'
  
  # load from s3
  S3.loadFile(key = 'test', path = '/tmp/loaded',bucket=s3bucketName)
  with open(path, 'r') as f:
    data = json.load(f)
#   print(data)
  # save to dynamodb
  results = []
  with ExampleData.batch_write() as batch:
    for item in data:
      exampleData = ExampleData.fromDict(item)
      print(exampleData.data)
      batch.save(exampleData)
  return results
    
  
  

In [23]:
ingestInput = {
  'key': 'test'
}
ingest(Event.getInput(ingestInput))

{'id_': '123', 'date': 1234323536, 'merchant_id': '32434', 'merchant_name': 'kdsjfljsldjf', 'transaction_id': 'qfeef', 'amount': 'qefqf', 'transaction_type': 'feqfe', 'consumer_name': 'fewqfq', 'mobile': 'qfefeq', 'terminal_id': 'qfewqf', 'transaction_date_time': 'fqef'}
{'id_': '123234', 'date': 1234323536, 'merchant_id': '32434', 'merchant_name': 'kdsjfljsldjf', 'transaction_id': 'qfeef', 'amount': 'qefqf', 'transaction_type': 'feqfe', 'consumer_name': 'fewqfq', 'mobile': 'qfefeq', 'terminal_id': 'qfewqf', 'transaction_date_time': 'fqef'}


[]

In [24]:
idList = ['123','123234']

In [26]:
for item in ExampleData.batch_get(idList):
    print(item.data)

{'id_': '123234', 'date': 1234323536, 'merchant_id': '32434', 'merchant_name': 'kdsjfljsldjf', 'transaction_id': 'qfeef', 'amount': 'qefqf', 'transaction_type': 'feqfe', 'consumer_name': 'fewqfq', 'mobile': 'qfefeq', 'terminal_id': 'qfewqf', 'transaction_date_time': 'fqef'}
{'id_': '123', 'date': 1234323536, 'merchant_id': '32434', 'merchant_name': 'kdsjfljsldjf', 'transaction_id': 'qfeef', 'amount': 'qefqf', 'transaction_type': 'feqfe', 'consumer_name': 'fewqfq', 'mobile': 'qfefeq', 'terminal_id': 'qfewqf', 'transaction_date_time': 'fqef'}
